In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 90.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/kaggle/QIA2023"

/content/drive/MyDrive/kaggle/QIA2023


In [ ]:

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from transformers import *
import os
import sys
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import numpy as np
import re
import pickle
import time
import pandas as pd
from pathlib import Path
import random
from torch.utils.tensorboard import SummaryWriter

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attenti

# 1. Test data Embedding 

In [ ]:
# load datasets
df = pd.read_csv('./data/test_data.csv')


In [ ]:
df['Gender'] = df['Gender'].map({'female':1,'male':0})
df

,Data_ID,User_ID,Gender,Age,Q_number,Short_Answer,Long_Answer
0,1,1,1,30,1,아니다,친구를 만들 상황에 새로운 친구를 만듭니다. 의도적으로나 꼭 주기적으로 새로운 친구...
1,2,1,1,30,2,그렇다,일상이 무료한 걸 못 참는 편. 요즘은 리본으로 뜨개질해서 가방 만드는 걸 배우고 ...
2,3,1,1,30,3,아니다,다른 사람이 울고 있는 모습을 본다고 해도 눈물이 나지는 않습니다. 그 심정은 이해...
3,4,1,1,30,4,그렇다,진행하고 있는 일이 계획단계일 때 이미 대비책을 함께 세우는 편. 잘못된 순간 바로...
4,5,1,1,30,5,그렇다,"최근 프로젝트 진행 중 문제가 발생, 마감기한이 얼마 남지 않았고, 주변 사람들도 ..."
...,...,...,...,...,...,...,...
7195,12776,213,1,20,56,아니다,저는 절차대로 일을 완수하는 편이 아닙니다. 이유는 절차적으로 일을 완수하는 것이 ...
7196,12777,213,1,20,57,아니다,저는 논쟁을 불러일으킬 주제에 관심이 없습니다. 이유는 논쟁을 불러일으킬 만한 주제...
7197,12778,213,1,20,58,아니다,저는 다른 사람에게 더 필요한 기회라고 생각되면 기회를 포기할 수 없습니다. 이유는...
7198,12779,213,1,20,59,아니다,저는 마감 기한을 지키기 쉽습니다. 이유는 주어진 기한 안에 문제를 해결하기 때문입...


In [ ]:
# load pretrained model
def get_model():
    # * Model          | Tokenizer          | Pretrained weights shortcut
    # MODEL=(DistilBertModel, DistilBertTokenizer, 'distilbert-base-uncased')
    tokenizer = BertTokenizerFast.from_pretrained("klue/bert-base")
    model = BertModel.from_pretrained("klue/bert-base")
    n_hl = model.config.num_hidden_layers
    return model, tokenizer, n_hl

In [ ]:
model, tokenizer, n_hl = get_model()

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/tokenizer_config.json


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/config.json
Model config BertConfig {
  "architectu

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/pytorch_model.bin
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification mo

In [ ]:
class MyMapDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        data = {k:v[idx] for k,v in self.data.items()}
        return data

In [ ]:
df['Long_Answer'] = df['Long_Answer'].str.strip()

In [ ]:
deliminator = pd.Series([' ' for i in range(len(df['Long_Answer']))])

df['Answer_added'] = df['Long_Answer'].add(deliminator) 
df['Answer_added'] = df['Answer_added'].add(df['Age'].astype('str')) 
df['Answer_added'] = df['Answer_added'].add(deliminator) 
df['Answer_added'] = df['Answer_added'].add(df['Gender'].astype('str'))
df['Answer_added'][0]

'친구를 만들 상황에 새로운 친구를 만듭니다. 의도적으로나 꼭 주기적으로 새로운 친구를 만들지는 않습니다. 30 1'

In [ ]:
tensor = tokenizer(df['Answer_added'].to_list(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)
ds = MyMapDataset(tensor)
dl = DataLoader(ds, batch_size=32, shuffle=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
def forward(model, dl, device=0):
    pooled = []
    hidden = []
    model.cuda(device)
    model.eval()
    for data in dl:
        data = {k:v.cuda(device) for k,v in data.items()}
        with torch.no_grad():
            output = model(**data, output_hidden_states=True)
        p, h = output.pooler_output, output.hidden_states
        pooled.append(p) # pooler output
        hidden.append(h[-1][:,0,:]) # only [CLS] token embedding 
    return torch.cat(pooled), torch.cat(hidden)

In [ ]:
result = forward(model, dl, device=0)

In [ ]:
result[0].shape, result[1].shape

(torch.Size([2880, 768]), torch.Size([2880, 768]))

In [ ]:
# forward datasets
# class MyDataset(Dataset):
#     def __init__(self, data):
#         self.data = data
#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         return self.data[idx]
    
# ds = MyDataset(result[0])
# dl = DataLoader(ds, shuffle=False, batch_size=)

# Load each model

In [ ]:
ie_path = './ckpt_only_p2/IE/epoch_500.pt'
sn_path = './ckpt_only_p2/SN/epoch_512.pt'
tf_path = './ckpt_only_p2/TF/epoch_236.pt'
jp_path = './ckpt_only_p2/JP/epoch_376.pt'
path = [ie_path, sn_path, tf_path, jp_path]

In [ ]:
def softmax(x):
    """
    Compute the softmax of array x.

    Parameters:
    x (numpy.ndarray): Input array of shape (n,).

    Returns:
    numpy.ndarray: Softmax values of shape (n,).
    """
    # Subtracting the maximum value to avoid numerical instability
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)

    return softmax_x

In [ ]:
def main(path):
    model = torch.load(path, map_location='cpu')
    model.eval()
    model = model.cuda(0)
    with torch.no_grad():
        output = model(result[0].cuda(0))
    output = output.tolist()
    for i in range(len(output)):
      output[i] = softmax(output[i])
    output = [row[1] for row in output]
        #output = output.argmax(axis=1)
    
    return output

In [ ]:
ie = main(ie_path)
sn = main(sn_path)
tf = main(tf_path)
jp = main(jp_path)

In [ ]:
test = pd.DataFrame({'User_ID': list(df['User_ID']),
                    'I/E': ie, 
                    'S/N': sn, 
                    'T/F': tf, 
                    'J/P': jp
                    })

In [ ]:
test = test.groupby('User_ID',as_index=False).mean()
test

,User_ID,I/E,S/N,T/F,J/P
0,1,0.621561,0.601044,0.432188,0.538886
1,2,0.534493,0.448594,0.669526,0.591750
2,3,0.599094,0.482652,0.617937,0.214143
3,4,0.973016,0.273584,0.529831,0.794734
4,5,0.840027,0.318410,0.422462,0.502871
...,...,...,...,...,...
115,175,0.135245,0.391633,0.883178,0.070841
116,178,0.021142,0.186231,0.714085,0.902669
117,179,0.449902,0.580717,0.287717,0.558586
118,191,0.682166,0.278361,0.199504,0.688742


In [ ]:
test.to_csv('basemodel_result.csv',index=False)